# Programming Exercise 8: Recommender systems

Welcome to part 2 of programming exercise 8, Recommendor systems. In this part of the exercise, we will implement the collaborative filtering learning algorithm and apply it to a dataset of movie ratings. This dataset consists of ratings on a scale of 1 to 5. The dataset has ${n_u = 943}$ users, and ${n_m = 1682}$ movies. The objective of collaborative filtering is to predict movie ratings for the movies that users have not yet rated, that is, the entries with ${R_{(i,j)} = 0}$. This will allow us to recommend the movies with the highest predicted ratings to the user.

**Instruction:**
- To run code in a cell: Do - `Shift+Enter` or Click - `Cell -> Run Cell`.

**Objective:**
- To learn how to implement the collaborative filtering learning algorithm and apply it to a dataset of movie ratings.

**You will learn how to:**
- Implement the collaborative filtering learning algorithm.

## Import packages ##

First lets run the cell below to import all the packages that you will need for this exercise.
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python
- [Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html) is a common library to load `.mat` files in Python.
- [matplotlib](http://matplotlib.org) is a common library to plot graphs in python.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import scipy.io as sio

# To make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# To suppress warnings
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

##  2.1 - Loading movie ratings dataset ##

We shall first load the dataset `./data/ex8_movies.mat`, that is providing the variables $Y$ and $R$. The matrix $Y$ (a num movies x num users matrix) stores the ratings $y^{(i,j)}$ (from 1 to 5). The matrix $R$ is an binary-valued indicator matrix, where $R{(i,j)} = 1$ if user $j$ gave a rating to movie $i$, and $R{(i,j)} = 0$ otherwise. The objective of collaborative filtering is to predict movie ratings for the movies that users have not yet rated, that is, the entries with $R{(i,j)} = 0$. This will allow us to recommend the movies with the highest predicted ratings to the user.

In [2]:
df_path = 'data/ex8_movies.mat' 
data    = sio.loadmat(df_path)
Y = data['Y']
R = data['R']

In this part of the exercise, we will be working with the matrices, $X$ and $Theta$:

![colfil](images/colfil.png)

The ${i-th}$ row of X corresponds to the feature vector $x^{(i)}$ for the ${i-th}$ movie, and the ${j-th}$ row of Theta corresponds to one parameter vector $\theta^{(j)}$, for the j-th user. Both $x^{(i)}$ and $\theta^{(j)}$ are n-dimensional vectors. For the purposes of this exercise, you will use $n = 100$, and therefore, $x^{(i)} \in \mathcal{R}^{100}$ and $\theta^{(j)} \in \mathcal{R}^{100}$. Correspondingly, $X$ is a ${n_m x 100}$ matrix and Theta is a $n_u x 100$ matrix.

**Results check**

In [3]:
print('\ni.  Y is a %d x %d matrix which stores the ratings(from 1 to 5) of %d movies on %d users.\n'\
      %(Y.shape[0],Y.shape[1],Y.shape[0],Y.shape[1]))
print('ii. R is a %d x %d binary-valued indicator matrix, where R(i,j) = 1\
if user j gave a rating to movie i,\n and R(i,j) = 0 otherwise\n' %(R.shape[0],R.shape[1]))


i.  Y is a 1682 x 943 matrix which stores the ratings(from 1 to 5) of 1682 movies on 943 users.

ii. R is a 1682 x 943 binary-valued indicator matrix, where R(i,j) = 1if user j gave a rating to movie i,
 and R(i,j) = 0 otherwise



In [4]:
#  From the matrix, we can compute statistics like average rating.
print('\nAverage rating for movie 1 (Toy Story): %f' % (Y[0,:].mean(axis=0)));


Average rating for movie 1 (Toy Story): 1.858961


## 2.2 - Collaborative filtering cost function ##

Now, we will start implementing the collaborative filtering learning algorithm. We will start by implementing the cost function (without regularization).

The collaborative filtering algorithm in the setting of movie recommendations considers a set of n-dimensional parameter vectors ${x^{(1)}...x^{(n_m)}}$ and $\theta^{(1)}...\theta^{(n_u)}$ where the model predicts the rating for movie $i$ by user $j$ as ${y^{(i,j)} = (\theta^{(j)})^Tx^{(i)}}$. Given a dataset that consists of a set of ratings produced by some users on some movies, you wish to learn the parameter vectors ${x^{(1)}...x^{(n_m)}}$, $\theta^{(1)}...\theta^{(n_u)}$ that produce the best fit minimizes the squared error).

In this part of the exercise, we shall be a provided `.mat` file having `X` and pretrained parameters `Theta`. So in the next cell we are loading them from `'data/ex8_movieParams.mat'`. 

In [5]:
# Load pre-trained weights (X, Theta, num_users, num_movies, num_features)
param_path = 'data/ex8_movieParams.mat' 
param_data    = sio.loadmat(param_path)
X     = param_data['X']
Theta = param_data['Theta']

Reduce the data set size so that this runs faster ..

In [6]:
num_users = 4; num_movies = 5; num_features = 3;
X     = X[0:num_movies, 0:num_features];
Theta = Theta[0:num_users, 0:num_features];
Y     = Y[0:num_movies, 0:num_users];
R     = R[0:num_movies, 0:num_users];
params = np.hstack((X.flatten(),Theta.flatten()))

### 2.2.1 - Collaborative filtering cost function ###

The collaborative filtering cost function (without regularization) is given by:
![cost_function](images/cost_function.png)

The cost function for collaborative filtering with regularization is given by:
![reg_costfunction](images/reg_costfunction.png)

In [7]:
def cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda):
    # Unfold the X and Theta matrices from params
    X = np.reshape(params[0:num_movies*num_features], (num_movies, num_features));
    Theta = np.reshape(params[num_movies*num_features:], (num_users, num_features));

    # Computing the cost
    pred = X.dot(Theta.T);
    error = pred - Y;
    error_factor = np.multiply(error,R);
    sqr_error = error_factor**2;
    total = np.sum(sqr_error);
    reg_term1 = (Lambda/2.) * np.sum(np.square(X))
    reg_term2 = (Lambda/2.) * np.sum(np.square(Theta))
    unreg_cost = (1/2) * total;                                  # Unregularised cost      
    J = unreg_cost + reg_term1 + reg_term2;                      # Regularized cost
    return J

### 2.2.2 - Collaborative filtering gradient ###

Now, you should implement the gradient (without regularization). Note that X grad should be a matrix of the same size as X and similarly, Theta grad is a matrix of the same size as Theta. 
The gradients of the cost function is given by:
![gradient](images/gradient.png)

Note that the gradients for the regularized cost function is given by:
![reg_gradient](images/reg_gradient.png)

In [8]:
def cofiGradFunc(params, Y, R, num_users, num_movies, num_features, Lambda):
    # Unfold the X and W matrices from params
    X = np.reshape(params[0:num_movies*num_features], (num_movies, num_features));
    Theta = np.reshape(params[num_movies*num_features:], (num_users, num_features));

    # Computing the gradient
    pred = X.dot(Theta.T);
    error = pred - Y;
    error_factor = np.multiply(error,R);
    sqr_error = error_factor**2;
    reg_x_grad = X * Lambda
    reg_theta_grad = Theta * Lambda
    X_grad = error_factor.dot(Theta) + reg_x_grad;              #regularised X gradients
    Theta_grad = error_factor.T.dot(X) + reg_theta_grad;        #regularised Theta gradients
    grads = np.hstack((X_grad.flatten(), Theta_grad.flatten())) # Regularized gradients
    return grads

Now we can evaluate the function we have defined for both unregularized and regularized cost function ...

In [9]:
#  Evaluate cost function
unregularized_j     = cofiCostFunc(params, Y, R, num_users, num_movies,num_features, 0.0);
unregularised_grads = cofiGradFunc(params, Y, R, num_users, num_movies,num_features, 0.0);
regularized_j       = cofiCostFunc(params, Y, R, num_users, num_movies,num_features, 1.5);
regularized_grads   = cofiGradFunc(params, Y, R, num_users, num_movies,num_features, 1.5);
print('\nCost at loaded parameters(lambda = 0.0): %f \
         \n(this value should be about 22.22)\n'%(unregularized_j));
print('Cost at loaded parameters(lambda = 1.5): %f \
         \n(this value should be about 31.34)\n'% (regularized_j));


Cost at loaded parameters(lambda = 0.0): 22.224604          
(this value should be about 22.22)

Cost at loaded parameters(lambda = 1.5): 31.344056          
(this value should be about 31.34)



## 2.3 - Entering ratings for a new user 

In [10]:
def loadMovieList():
    movie_list =[]
    with open("./data/movie_ids.txt") as f:
        for line in f:
            movie_list.append(line[line.index(' ') + 1:].rstrip())
    return movie_list

In [11]:
movie_list = loadMovieList();

In [12]:
# Initialize my ratings
my_ratings = np.zeros((len(movie_list), 1));

In [13]:
# Check the file movie_idx.txt for id of each movie in our dataset
# For example, Toy Story (1995) has ID 1, so to rate it "4", you can set
my_ratings[1] = 4;
my_ratings[98] = 2;
my_ratings[7] = 3;
my_ratings[12]= 5;
my_ratings[54] = 4;
my_ratings[64]= 5;
my_ratings[66]= 3;
my_ratings[69] = 5;
my_ratings[183] = 4;
my_ratings[226] = 5;
my_ratings[355]= 5;

In [14]:
print('\nNEW USER RATINGS:\n');
for i in range(len(my_ratings)):
    if my_ratings[i] > 0: 
        print('\tRated %d for %s\n' % (my_ratings[i],movie_list[i]));


NEW USER RATINGS:

	Rated 4 for GoldenEye (1995)

	Rated 3 for Babe (1995)

	Rated 5 for Mighty Aphrodite (1995)

	Rated 4 for Professional, The (1994)

	Rated 5 for What's Eating Gilbert Grape (1993)

	Rated 3 for Ace Ventura: Pet Detective (1994)

	Rated 5 for Four Weddings and a Funeral (1994)

	Rated 2 for Snow White and the Seven Dwarfs (1937)

	Rated 4 for Army of Darkness (1993)

	Rated 5 for Star Trek VI: The Undiscovered Country (1991)

	Rated 5 for Client, The (1994)



## Part 7: Learning Movie Ratings 

 Now, you will train the collaborative filtering model on a movie rating dataset of 1682 movies and 943 users

In [15]:
#  Import and load movie data
#  Y is a 1682x943 matrix, containing ratings (1-5) of 1682 movies by 943 users
#  R is a 1682x943 matrix, where R(i,j) = 1 if and only if user j gave a rating to movie i
movie_path    = 'data/ex8_movies.mat' 
movie_data    = sio.loadmat(movie_path)
Y = movie_data['Y']
R = movie_data['R']

In [16]:
#  Add our own ratings to the data matrix
# Add our own ratings to the data matrix
Y = np.hstack((my_ratings.reshape(len(movie_list), 1), Y))
R = np.hstack((my_ratings.reshape(len(movie_list), 1) != 0, R))
#Y = np.c_[my_ratings, Y]; 
#R = np.c_[((my_ratings != 0).astype(int).reshape(-1,1)!= 0),R];

In [17]:
# Function to normalize ratings
def normalizeRatings(Y, R):
    (m, n) = Y.shape;
    Ymean = np.zeros((m, 1));
    Ynorm = np.zeros(Y.shape);
    for i in range(m):
        idx = np.nonzero(R[i, :] == 1);
        Ymean[i] = Y[i, idx].mean(axis=1);
        Ynorm[i, idx] = Y[i, idx] - Ymean[i];
    return Ynorm, Ymean

In [18]:
Ynorm, Ymean = normalizeRatings(Y, R)

In [19]:
#  Useful Values
num_users = Y.shape[1];
num_movies = Y.shape[0];
num_features = 10;

In [20]:
# Set Initial Parameters (Theta, X)
X     = np.random.randn(num_movies, num_features);
Theta = np.random.randn(num_users, num_features);
initial_parameters = np.hstack((X.flatten(), Theta.flatten()));
import scipy.optimize as opt

In [21]:
def train_collaborative_filtering(Ynorm, R, num_users, num_movies, num_features, Lambda):
    # Set Initial Parameters (Theta, X)
    X     = np.random.randn(num_movies, num_features);
    Theta = np.random.randn(num_users, num_features);
    initial_parameters = np.hstack((X.flatten(), Theta.flatten()));
    
    # create cost and grad functions
    cost = lambda p: cofiCostFunc(p, Ynorm, R, num_users, num_movies,num_features, Lambda);
    grad = lambda p: cofiGradFunc(p, Ynorm, R, num_users, num_movies,num_features, Lambda).flatten();
    
    # minimize using fmincg
    result = opt.fmin_cg(cost, initial_parameters.T, fprime=grad, maxiter=500, disp=True)

    return result 

In [22]:
Lambda = 10
result = train_collaborative_filtering(Ynorm, R, num_users, num_movies, num_features, Lambda)

         Current function value: 38965.031162
         Iterations: 500
         Function evaluations: 759
         Gradient evaluations: 759


In [23]:
X     = result[0:num_movies * num_features].reshape((num_movies, num_features))
Theta = result[num_movies * num_features:].reshape((num_users, num_features))

In [24]:
p = X.dot(Theta.T)
my_predictions = p[:,0]+ Ymean.ravel()
idx = my_predictions.argsort()[::-1]
print('\nTOP RECOMMENDATION FOR YOU:')
for i in range(10):
    j = idx[i]
    print('Predicting rating %.1s for movie %s'% (my_predictions[j], movie_list[j]))


TOP RECOMMENDATION FOR YOU:
Predicting rating 5 for movie Aiqing wansui (1994)
Predicting rating 5 for movie Santa with Muscles (1996)
Predicting rating 5 for movie Star Kid (1997)
Predicting rating 5 for movie Marlene Dietrich: Shadow and Light (1996)
Predicting rating 5 for movie Great Day in Harlem, A (1994)
Predicting rating 4 for movie Someone Else's America (1995)
Predicting rating 4 for movie Saint of Fort Washington, The (1993)
Predicting rating 4 for movie Entertaining Angels: The Dorothy Day Story (1996)
Predicting rating 4 for movie They Made Me a Criminal (1939)
Predicting rating 4 for movie Prefontaine (1997)


In [25]:
print('\nOriginal ratings provided:')
(index, _) = np.nonzero(my_ratings)
for i in range(len(index)):
    j = index[i]
    print('Rated %.1s for movie %s'% (my_ratings[j,0], movie_list[j]))


Original ratings provided:
Rated 4 for movie GoldenEye (1995)
Rated 3 for movie Babe (1995)
Rated 5 for movie Mighty Aphrodite (1995)
Rated 4 for movie Professional, The (1994)
Rated 5 for movie What's Eating Gilbert Grape (1993)
Rated 3 for movie Ace Ventura: Pet Detective (1994)
Rated 5 for movie Four Weddings and a Funeral (1994)
Rated 2 for movie Snow White and the Seven Dwarfs (1937)
Rated 4 for movie Army of Darkness (1993)
Rated 5 for movie Star Trek VI: The Undiscovered Country (1991)
Rated 5 for movie Client, The (1994)
